### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import os

# File to Load (Remember to Change These)
filepath = os.path.join("Resources","Heroes.csv")

# Read Purchasing File and store into Pandas data frame
purchase_df = pd.read_csv(filepath)

purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [3]:
# Total Number of Players (unique SNs)
unique = purchase_df['SN'].unique() 

# The length of the list of unique players will provide the total # of players
number_players = len(unique)

# Display the total number of players

print(f'Total Players: {number_players}')

test_df = pd.DataFrame({'Total Players':number_players},index = ['Count'])

test_df

Total Players: 576


,Total Players
Count,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
# Number of Unique items
unique_items = len(purchase_df['Item Name'].unique())

# Average Price
average_price = round(purchase_df['Price'].mean(),2)

# Total number of Purchases
total_purchases = purchase_df['Purchase ID'].count()

# Total Revenue
total_revenue = purchase_df['Price'].sum()

# Summary Dataframe + Reformatting

summary_table = [{"Number of Unique Items":unique_items,"Average Price":average_price,"Number of Purchases":total_purchases,"Total Revenue":total_revenue}]

summary_df = pd.DataFrame(summary_table)

summary_df['Average Price'] = summary_df['Average Price'].map('${:,.2f}'.format)

summary_df['Total Revenue'] = summary_df['Total Revenue'].map('${:,.2f}'.format)

summary_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
male = 0
female = 0
other = 0
SN_list = []

# Calculate number of players by Gender
for i, row in purchase_df.iterrows():
    if row[3] == 'Male' and row[1] not in SN_list:
        male = male + 1
        SN_list.append(row[1])
    elif row[3] == 'Female' and row[1] not in SN_list:
        female = female + 1
        SN_list.append(row[1])
    elif row[3] == 'Other / Non-Disclosed' and row[1] not in SN_list:
        other = other + 1
        SN_list.append(row[1])

# Calculate percentage of players by Gender

male_pct = round(male/len(unique),4)
female_pct = round(female/len(unique),4)
other_pct = round(other/len(unique),4)

# Create summary dataframe

gender_summary = {'Total Count':[male,female,other],'Percentage of Players':[male_pct,female_pct,other_pct]}

gender_summary_df = pd.DataFrame(gender_summary,index = ['Male','Female','Other / Non-Disclosed'])
gender_summary_df['Percentage of Players'] = gender_summary_df['Percentage of Players'].map("{:.2%}".format)

gender_summary_df.reset_index(inplace=True)

gender_summary_df


# LATER REALIZED I COULD USE .NUNIQUE - on a groupby for gender

        

,index,Total Count,Percentage of Players
0,Male,484,84.03%
1,Female,81,14.06%
2,Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
# Run .groupby on purchase_df to group by the Gender
grouped_df = purchase_df.groupby(['Gender'])


# Calculate Purchase count by Gender
x = grouped_df['Item ID'].count()



# Calculate Average Purchase Price by Gender
y = round(grouped_df['Price'].mean(),2)



# Calculate Total Purchase Value by Gender

z = grouped_df['Price'].sum()


# Create summary dataframe of x,y,z series values above
summary_df = pd.DataFrame({"Purchase Count":x,"Average Purchase Price":y,"Total Spent":z}).reset_index()


# Merge summary_df with gender_summary_df based upon Gender column

    # Rename index to Gender

gender_summary_df.rename(columns = {'index':'Gender'}, inplace = True)



merged_df = pd.merge(summary_df,gender_summary_df,on='Gender')


merged_df["Average Total Purchase per Person"] = round(merged_df["Total Spent"]/merged_df["Total Count"],2)


# Reformat values in merged DataFrame
merged_df['Average Purchase Price'] = merged_df['Average Purchase Price'].map('${:,.2f}'.format)

merged_df['Average Total Purchase per Person'] = merged_df['Average Total Purchase per Person'].map('${:,.2f}'.format)

merged_df['Total Spent'] = merged_df['Total Spent'].map('${:,.2f}'.format)


#Re-order the columns into another DataFrame and set Gender as the Index
reorganized_df = merged_df[['Gender','Purchase Count','Average Purchase Price','Total Spent','Average Total Purchase per Person']]


reorganized_df.set_index('Gender',inplace=True)


reorganized_df



        

        


,Purchase Count,Average Purchase Price,Total Spent,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [7]:
# Create bins and associated labels
bins = [0,9.9,14.9,19.9,24.9,29.9,34.9,39.9,100000]
labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

# Add a column to the original purchase_df with the bins

purchase_df['Age Groups'] = pd.cut(purchase_df['Age'],bins=bins,labels=labels)

# Group by the newly created Age Groups column that was generated from the bins

grouped_ages = purchase_df.groupby(['Age Groups'])

# Count only the number of unique SNs in each bin 

unique_ages_grouped = grouped_ages['SN'].nunique()

# Create a DataFrame from the count of unique SNs per bin

purchase_bins_df = pd.DataFrame(unique_ages_grouped)

# Add a column for the Percentage of total players in each bin  to the dataframe and convert these values to a % format

purchase_bins_df['Percentage of Players'] = round(purchase_bins_df['SN']/unique_ages_grouped.sum(),4)

purchase_bins_df['Percentage of Players'] = purchase_bins_df['Percentage of Players'].map("{:.2%}".format)

# Rename the 'SN' column to 'Count' and display the resulting DataFrame

purchase_bins_df = purchase_bins_df.rename(columns = {'SN':'Count'})

purchase_bins_df




,Count,Percentage of Players
Age Groups,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
# Calculate Purchase Counts by Age group

Total_Purchases_Age = grouped_ages['Purchase ID'].count()

# Calculate Average Purchase Price by Age Group

Average_PurPrice_Age = round(grouped_ages['Price'].mean(),2)

# Calculate Total Purchase Value by Age Group

Total_Value_Age = grouped_ages['Price'].sum()

# Calculate Average Purchase Price per Person by Age Group

Avg_PPP_Age = round(grouped_ages['Price'].sum()/grouped_ages['SN'].nunique(),2)

# Create a summary DataFrame of all calculations above

age_summary_df = pd.DataFrame({'Purchase Count':Total_Purchases_Age,'Average Purchase Price':Average_PurPrice_Age,'Total Purchase Value':Total_Value_Age,'Avg Total Purchase per Person':Avg_PPP_Age}).reset_index()

# Set the index of the summary DataFrame as 'Age Groups' (the bins created previously)

age_summary_df.set_index('Age Groups',inplace=True)

# Format $$ columns in $ and display resulting DataFrame

age_summary_df['Total Purchase Value'] = age_summary_df['Total Purchase Value'].map('${:,.2f}'.format)

age_summary_df['Average Purchase Price'] = age_summary_df['Average Purchase Price'].map('${:,.2f}'.format)

age_summary_df['Avg Total Purchase per Person'] = age_summary_df['Avg Total Purchase per Person'].map('${:,.2f}'.format)

age_summary_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Groups,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
# Run a groupby on the original purchase dataframe to group by SN

grouped_sn = purchase_df.groupby('SN')

# Count the number of items per SN 

item_count_sn = grouped_sn['Item ID'].count()

# Sum on the price to get the total by SN

total_purchase_sn = grouped_sn['Price'].sum()

# Get the mean of the purchase price by SN

mean_purchase_sn = grouped_sn['Price'].mean()


# Create a new Dataframe with the results

spenders_df = pd.DataFrame({'Purchase Count':item_count_sn,'Average Purchase Price':mean_purchase_sn,'Total Purchase Value':total_purchase_sn})

# Sort the dataframe based upon Total Purchase Value (stored as a new dataframe [not necessary])

sorted_spenders_df = spenders_df.sort_values(by='Total Purchase Value',ascending=False)

# Reformat the Average Purchase Price and Total Purchase Value columns to display as $xx.xx and display the resulting DataFrame preview

sorted_spenders_df['Average Purchase Price'] = sorted_spenders_df['Average Purchase Price'].map('${:,.2f}'.format)

sorted_spenders_df['Total Purchase Value'] = sorted_spenders_df['Total Purchase Value'].map('${:,.2f}'.format)



sorted_spenders_df.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
# Run a groupby on the original purchase_df for Item Names & Item ID

grouped_items = purchase_df.groupby(['Item ID','Item Name'])

# Run a count on any column to get the total item counts in the group

item_counts = grouped_items['SN'].count()

# Run a mean on the price to get the average price in the group

item_costs = grouped_items['Price'].mean()

# Run a sum on the price to get the Total Purchase Value

item_totval = grouped_items['Price'].sum()

# Create a summary data frame based on the calculations above

summary_items_df = pd.DataFrame({'Purchase Count':item_counts,'Item Price':item_costs,'Total Purchase Value':item_totval})

# Sort the summary_items_df by Purchase Count

sorted_items_df = summary_items_df.sort_values('Purchase Count',ascending=False)

# Reformat the $ values in the dataframe and display the resulting DataFrame preview

sorted_items_df['Item Price'] = summary_items_df['Item Price'].map('${:,.2f}'.format)

sorted_items_df['Total Purchase Value'] = summary_items_df['Total Purchase Value'].map('${:,.2f}'.format)



sorted_items_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [11]:
# Sort the above table by Total Purchase Value and store as a separate DataFrame

most_profitable_df = summary_items_df.sort_values('Total Purchase Value',ascending = False)

most_profitable_df['Total Purchase Value'] = most_profitable_df['Total Purchase Value'].map('${:,.2f}'.format)

most_profitable_df['Item Price'] = most_profitable_df['Item Price'].map('${:,.2f}'.format)

most_profitable_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
